In [28]:
# -*- coding: utf-8 -*-
instructions = [
    "/msralaphilly2/ml-la/renqian/SFM/threedimargen/data/materials_data/instruct/base1b/instruct_task_20240807/1b_dialogue_1v1_bs2048_steps_20000/all/test.comp_to_material.tsv.response.valid.inst.txt",
    "/msralaphilly2/ml-la/renqian/SFM/threedimargen/data/materials_data/instruct/base8b/instruct_task_20240807/8b_dialogue_1v1_bs2048_steps_20000/all/test.comp_to_material.tsv.response.valid.inst.txt",
    "/msralaphilly2/ml-la/renqian/SFM/threedimargen/data/materials_data/instruct/8x7b/7755/test.comp_to_material.response.valid.inst.txt.0.75"
]

responses = [
    "/msralaphilly2/ml-la/renqian/SFM/threedimargen/data/materials_data/instruct/base1b/instruct_task_20240807/1b_dialogue_1v1_bs2048_steps_20000/all/test.comp_to_material.tsv.response.valid.txt",
    "/msralaphilly2/ml-la/renqian/SFM/threedimargen/data/materials_data/instruct/base8b/instruct_task_20240807/8b_dialogue_1v1_bs2048_steps_20000/all/test.comp_to_material.tsv.response.valid.txt",
    "/msralaphilly2/ml-la/renqian/SFM/threedimargen/data/materials_data/instruct/8x7b/7755/test.comp_to_material.response.valid.txt.0.75",
]

In [40]:
train_file = "/msralaphilly2/ml-la/yinxia/wu2/shared/SFM/SFM.overall.data/SFMMolInstruct.20240807/BulkModulus/train.bulk.tsv"
import pandas as pd
from collections import Counter
train_data = []
with open(train_file, "r") as f:
    data = pd.read_csv(f, sep="\t", header=None)
    for row in data.iterrows():
        inst = row[1][0]
        seq = row[1][1]
        seq = seq.replace("<material>", "").replace("</material>", "").strip().split()
        elements = seq[:-1]
        sg = seq[-1]
        elements = Counter(elements)
        formula = ""
        for key in sorted(elements.keys()):
            formula += f"{key}{elements[key]}"
        formula += sg
        train_data.append(formula)
print(len(train_data))
print(train_data[:10])
train_data = set(train_data)
print(len(train_data))

42693
['Bi2Ir1Pd1<sg164>', 'Nd2Sn2<sg63>', 'Ag1As2Mn1<sg166>', 'Sb2Sc2<sg109>', 'B3C5U4<sg25>', 'As2Sc1Sn1<sg164>', 'Bi3Th1<sg123>', 'Ac1Eu2<sg191>', 'Ge2Hf2<sg63>', 'Ca1Hg2<sg139>']
35093


In [41]:
from collections import Counter
import re
import periodictable


chemical_elements = [element.symbol for element in periodictable.elements][1:]

pattern = re.compile(r'\b(' + '|'.join(chemical_elements) + r')\b')  


file_indices = {
    0: "SFM-Seq 1B",
    1: "SFM-Seq 8B",
    2: "SFM-Seq 8x7B"
}


def evaluate(index, novelty=False):
    metrics = []
    total = 0
    novelty = 0
    with open(instructions[index], "r") as f:
        insts = f.readlines()
    with open(responses[index], "r") as f:
        resps = f.readlines()
    for inst, resp in zip(insts, resps):
        seq = resp.split()[:-1]
        sg = resp.split()[-1]
        temp = Counter(seq)
        formula = ""
        for key in sorted(temp.keys()):
            formula += f"{key}{temp[key]}"
        formula += sg

        total += 1
        if novelty and (formula in train_data):
            continue
        novelty += 1

        # determine elements in inst
        elements_in_inst = set(pattern.findall(inst))
        if len(elements_in_inst) == 0:
            continue
        # determine elements in seq
        elements_in_seq = set(resp.split()[:-1])
        # determine elements appeard in both
        elements_in_both = set(elements_in_inst) & set(elements_in_seq)
        metric = len(set(elements_in_both)) / len(set(elements_in_inst))
        metrics.append(metric)

    print(f"{file_indices[index]} {sum(metrics) / len(metrics)}")
    if novelty:
        print(f"Total: {total}")
        print(f"Novel: {novelty}")
        print(f"Novel rate: {novelty}/{total} = {novelty/total:.4f}")
    return       
                

In [36]:
evaluate(0)
evaluate(1)
evaluate(2)

SFM-Seq 1B 0.9794822554712099
SFM-Seq 8B 0.9844021495745603
SFM-Seq 8x7B 0.9767859469641964


In [42]:
evaluate(0, True)
evaluate(1, True)
evaluate(2, True)

SFM-Seq 1B 0.9792372166906258
Total: 11069
Novel: 10751
Novel rate: 10751/11069 = 0.9713
SFM-Seq 8B 0.9838287871150727
Total: 12792
Novel: 12218
Novel rate: 12218/12792 = 0.9551
SFM-Seq 8x7B 0.9759970097066786
Total: 25631
Novel: 24306
Novel rate: 24306/25631 = 0.9483
